# QSVM

In [1]:
import pandas as pd
from qiskit import Aer
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit.circuit.library import ZZFeatureMap
from qiskit_machine_learning.kernels import QuantumKernel, FidelityQuantumKernel
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [2]:
df = pd.read_csv('datasets/vlds.csv', index_col=0)
df.head(3)

,feature_1,feature_2,feature_3,feature_4,label
0,7.0,19.0,9.0,8.0,1.0
1,8.0,22.0,15.0,4.0,1.0
2,5.0,26.0,19.0,14.0,1.0


In [3]:
feature_dimension = df.shape[1] - 1
print(f"Feature dimension: {feature_dimension}")

Feature dimension: 4


In [4]:
# Split the data into training and test sets
train = df.sample(frac=0.75, random_state=42)
test = df.drop(train.index)

# Separate the features from the labels
train_features = train.copy()
test_features = test.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

In [5]:
algorithm_globals.random_seed = 42
quantum_instance = QuantumInstance(Aer.get_backend("aer_simulator"), shots=1024)

In [6]:
feature_map = ZZFeatureMap(feature_dimension=feature_dimension, reps=2, entanglement='linear')
kernel = QuantumKernel(feature_map=feature_map, quantum_instance=quantum_instance)
# kernel = FidelityQuantumKernel(fidelity=ComputeUncompute(sampler=Sampler()))
svc = SVC(kernel=kernel.evaluate)

In [7]:
svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)
print(f"QSVM score: {score}")

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


QSVM score: 0.68


In [8]:
scores = cross_val_score(svc, train_features, train_labels, cv=10)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/l

[0.60526316 0.55263158 0.60526316 0.60526316 0.60526316 0.67567568
 0.64864865 0.56756757 0.64864865 0.62162162]
0.61 accuracy with a standard deviation of 0.04


In [9]:
classical_svc = SVC()
classical_svc.fit(train_features, train_labels)
score = svc.score(test_features, test_labels)

/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/lukas/miniforge3/envs/qc/lib/python3.9/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


In [10]:
print(f"Classical SVC score: {score}")

Classical SVC score: 0.656


In [11]:
scores = cross_val_score(classical_svc, train_features, train_labels)
scores

array([0.93333333, 1.        , 0.97333333, 0.98666667, 0.97333333])

In [12]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))


0.97 accuracy with a standard deviation of 0.02
